In [ ]:
using Pkg
Pkg.activate(joinpath(@__DIR__, ".."))
Pkg.instantiate()

In [ ]:
using Revise
using TORA

In [ ]:
using MeshCat
using MeshCatMechanisms
using RigidBodyDynamics

In [ ]:
TORA.greet()

In [ ]:
vis = Visualizer()

setprop!(vis["/Cameras/default/rotated/<object>"], "fov", 40)

# IJuliaCell(vis)  # Show the viewer here (in the notebook)
open(vis)  # Show the viewer in a separate tab

In [ ]:
robot = TORA.create_robot_kuka_iiwa_14(vis)
problem = TORA.Problem(robot, 51, 0.020)

# Constrain initial and final joint velocities to zero
TORA.fix_joint_velocities!(problem, robot, 1, zeros(robot.n_v))
TORA.fix_joint_velocities!(problem, robot, problem.num_knots, zeros(robot.n_v))

# Constrain the position of the end-effector
let
    CubicTimeScaling(Tf::Number, t::Number) = 3(t / Tf)^2 - 2(t / Tf)^3
    QuinticTimeScaling(Tf::Number, t::Number) = 10(t / Tf)^3 - 15(t / Tf)^4 + 6(t / Tf)^5

    for k = 1:1:problem.num_knots
        θ = CubicTimeScaling(problem.num_knots, k) * 2π
        pos = [0.5, 0.2 * sin(θ), 1.0 + 0.2 * cos(θ)]
        # pos = [0.5, 0.3 * sin(θ) + 0.1 * sin(8 * θ), 0.8 + 0.3 * cos(θ) + 0.1 * cos(8 * θ)]
        TORA.constrain_ee_position!(problem, robot, k, pos)
    end
end

# TORA.constrain_ee_position!(problem, robot,   1, [ 1.0,  0.0,  0.5])
# foreach(k -> TORA.constrain_ee_position!(problem, robot, k, [ 0.0,  1.0,  0.5]), 30:50)
# foreach(k -> TORA.constrain_ee_position!(problem, robot, k, [-1.0,  0.0,  0.5]), 60:80)
# TORA.constrain_ee_position!(problem, robot, 101, [ 0.0, -1.0,  0.5])

TORA.show_problem_info(problem)

In [ ]:
cpu_time, x = TORA.solve_with_knitro(problem, robot)
# cpu_time, x = TORA.solve_with_ipopt(problem, robot)

In [ ]:
let
    ee_positions = TORA.get_ee_path(problem, robot, x)
    TORA.show_ee_path(vis, ee_positions)

    nₓ = robot.n_q + robot.n_v + robot.n_τ  # dimension of each mesh point
    ind_q = hcat([range(1 + (i * nₓ), length=robot.n_q) for i = (1:problem.num_knots) .- 1]...)
    q_mat = x[ind_q]

    ts = range(0, length=problem.num_knots, step=problem.dt)
    qs = [q_mat[:, i] for i = 1:size(q_mat, 2)]

    animation = Animation(robot.mvis, ts, qs)
    setanimation!(robot.mvis, animation)
end

In [ ]:
rand_configuration!(robot.state)
set_configuration!(robot.mvis, configuration(robot.state))